In [1]:
import pandas as pd

In [2]:
excel_file = pd.ExcelFile('names.xlsx')
sheets = excel_file.sheet_names

# Concatenate all sheets into one table
df = pd.concat([excel_file.parse(sheet) for sheet in sheets])


In [3]:
df_subset = df.iloc[:, 1:3]

df_subset = df_subset[pd.to_numeric(df_subset.iloc[:, 0], errors='coerce').notnull()]

df_subset = df_subset.set_index(df_subset.columns[0])

# Step 1: Split the first column by space character
df_subset['split'] = df_subset.iloc[:, 0].str.split(' ')

# Step 2: Create a new column for the third string
df_subset['faculty'] = df_subset['split'].apply(lambda x: x[2] if len(x) > 2 else None)

# Step 3: Modify the original cell to only contain the first two strings
df_subset.iloc[:, 0] = df_subset['split'].apply(lambda x: ' '.join(x[:2]) if len(x) > 2 else ' '.join(x))

# Step 4: Drop the original split column
df_subset = df_subset.drop(columns=['split'])


In [4]:
df_subset['average'] = 0
max = 0
total = 0

In [5]:
def add_file(file_name, coef, column):
    coef = coef / 3 * 2
    global total
    total += coef
    excel_file = pd.ExcelFile(file_name)
    sheets = excel_file.sheet_names

    # Concatenate all sheets into one table
    df = pd.concat([excel_file.parse(sheet) for sheet in sheets])

    dsa = df.iloc[:, 1:]

    dsa = dsa[pd.to_numeric(dsa.iloc[:, 0], errors='coerce').notnull()]

    dsa = dsa.set_index(dsa.columns[0])

    for index, row in dsa.iterrows():
        if index in df_subset.index:
            value = row[column-3]
            if pd.isnull(value):
                continue
            if isinstance(value, str):
                value = value.replace(',', '.')
                try:
                    value = float(value)
                except ValueError as e:
                    print(value + " " + e.__str__())
                    continue
            df_subset.at[index, 'average'] += value / 20 * coef

In [6]:
# CS only
add_file('subjects/dsa.xlsx', 3, 3)
add_file('subjects/dsa.xlsx', 3, 4)
add_file('subjects/front.xlsx', 1, 5)
add_file('subjects/front-0.2.xlsx', 0.2, 3)

In [7]:
# everyone except CS
add_file('subjects/tom.xlsx', 0.75, 3)

In [8]:
add_file('subjects/electro.xlsx', 0.5, 5)
add_file('subjects/electro.xlsx', 1.5, 8)

In [9]:
add_file('subjects/chemeng.xlsx', 2, 3)

In [10]:
add_file('subjects/french.xlsx', 1.125, 5)
add_file('subjects/math.xlsx', 1, 3)
add_file('subjects/point.xlsx', 0.5, 5)
add_file('subjects/point.xlsx', 1.5, 8)
add_file('subjects/fluid.xlsx', 2, 3)

abs could not convert string to float: 'abs'


In [11]:
add_file('subjects/french.xlsx', 0.75, 3)
add_file('subjects/french.xlsx', 1.125, 4)
add_file('subjects/math.xlsx', 2, 4)
add_file('subjects/aom.xlsx', 0.75, 3)
print(total)

abs could not convert string to float: 'abs'
abs could not convert string to float: 'abs'
ABSENT could not convert string to float: 'ABSENT'
15.133333333333335


In [12]:
# faculty filter
df_subset = df_subset[df_subset['faculty'] == 'GE-22']

In [13]:
df_subset = df_subset.sort_values(df_subset.columns[-1])
df_subset['Rank'] = df_subset[df_subset.columns[-1]].rank(method='first', ascending=False)
df_subset = df_subset.sort_values('Rank')
df_subset = df_subset.set_index('Rank')
df_subset.index.name = None

In [14]:
df_subset.to_excel('output.xlsx')